# Scraping Wikipedia Tables
I am going to scrape a particular wikipedia page and extract a table and transform it into a pandas Dataframe. 

The task are the following: 
1. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
2. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
3. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
4. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
5. Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
6. In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe


### Notebook's summary: 
1. Create an empty dataframe df (**task1**)
2. Scrape the wiki page 
3. Populate the df and completing task (**taks2, task 4**)
4. Once you have the data in df, wrangle it (**task 3**)
5. Visualize the shape of the dataframe (**task6**)

## 1. Create an empty dataframe 

In [1]:
import pandas as pd 
import requests

In [2]:
columns_name = ["PostalCode","Borough", "Neighborhood" ]
df = pd.DataFrame(columns = columns_name)

## 2. Scrape the Wiki page 
I'm using beautifulsoup 

In [22]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())

## 3. Populate the dataframe 


by using the browser inspect element I can see that I need to focus on the table tag
Here I am going to drop the first `<tr>` tag since it is representing the headers of the table. 

- content represents the whole content of the table and row represent each row of the table 

- I am using try and except to avoid having any errors during the scraping (playing it safe) but in this particular case we could avoid it since the table is complete. 

- I am avoiding to process any rows that has Borough as Not Assigned. (**task2**) and while checking the neighborhood I am replacing it to `borough` value if it's not assigned.(**task4**) 

- Append everything to df dataframe 

In [5]:
table = soup.find('table',{'class':'wikitable sortable'})
content = table.find_all('tr')
del content[0]

for row in content: 
    element = row.find_all('td')
    if (element[1].text != 'Not assigned'): 
        try: 
            postalcode = element[0].text
        except: 
            postalcode = none
        try: 
            borough = element[1].text
        except: 
            borough = none
        try: 
            neighborhood = element[2].text.split("\n")[0]
            if (neighborhood == "Not assigned"): 
                neighborhood = borough
        except: 
            neighborhood = none


        df = df.append({"PostalCode":postalcode, 
                        "Borough": borough,
                        "Neighborhood": neighborhood}, ignore_index = True)
    else: 
        pass 

In [6]:
df.head(10)

PostalCode           Borough      Neighborhood
0        M3A        North York         Parkwoods
1        M4A        North York  Victoria Village
2        M5A  Downtown Toronto      Harbourfront
3        M5A  Downtown Toronto       Regent Park
4        M6A        North York  Lawrence Heights
5        M6A        North York    Lawrence Manor
6        M7A      Queen's Park      Queen's Park
7        M9A         Etobicoke  Islington Avenue
8        M1B       Scarborough             Rouge
9        M1B       Scarborough           Malvern

looks ok!

## 4. Wrangle the dataframe 
in order to complete the task 3, we need to check how many unique PostCode we have. 
Task 3: More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

Which postcodes have more than one neighborhood?

In [11]:
postcode = df.groupby("PostalCode")[["Neighborhood"]].count()
postcode.head()

Neighborhood
PostalCode              
M1B                    2
M1C                    3
M1E                    3
M1G                    1
M1H                    1

How many rows my final dataframe will need to have: 

In [12]:
df["PostalCode"].nunique()

103

Prepare a brand new dataframe for the final changes. 

In [14]:
columns_name = ["PostalCode","Borough", "Neighborhood" ]
df2 = pd.DataFrame(columns = columns_name)


I will use as a reference the table above which will guide me through how to change the dataframe. 

- If the Postcode has only one neighborhood, then just append the row from df to df2. 

- If the Postcode has more than one neighborhood, then reorder the index, and append to the first instance the remaining neighborhood. Finally append the first instance that contains all the neighborhoods. 

In [15]:

for index, row in postcode.iterrows():
    
    one_postcode = df.loc[df["PostalCode"] == index, ['PostalCode', 'Borough', 'Neighborhood']]
    one_postcode.reset_index(drop=True, inplace=True)
    
    if (len(one_postcode) > 1): 

        for i in range(1,len(one_postcode)): 
            one_postcode["Neighborhood"][0] = one_postcode["Neighborhood"][0] + ", " + one_postcode['Neighborhood'][i]
        first = one_postcode.iloc[[0]]
        first.reset_index(drop=True, inplace=True)
        df2 = df2.append(first, ignore_index = True)
        
    else: 
        df2 = df2.append(one_postcode, ignore_index = True)

In [21]:
df2

PostalCode           Borough  \
0          M1B       Scarborough   
1          M1C       Scarborough   
2          M1E       Scarborough   
3          M1G       Scarborough   
4          M1H       Scarborough   
5          M1J       Scarborough   
6          M1K       Scarborough   
7          M1L       Scarborough   
8          M1M       Scarborough   
9          M1N       Scarborough   
10         M1P       Scarborough   
11         M1R       Scarborough   
12         M1S       Scarborough   
13         M1T       Scarborough   
14         M1V       Scarborough   
15         M1W       Scarborough   
16         M1X       Scarborough   
17         M2H        North York   
18         M2J        North York   
19         M2K        North York   
20         M2L        North York   
21         M2M        North York   
22         M2N        North York   
23         M2P        North York   
24         M2R        North York   
25         M3A        North York   
26         M3B        North York   
27         M3C        North York   
28         M3H        North York   
29         M3J        North York   
..         ...               ...   
73         M6C              York   
74         M6E              York   
75         M6G  Downtown Toronto   
76         M6H      West Toronto   
77         M6J      West Toronto   
78         M6K      West Toronto   
79         M6L        North York   
80         M6M              York   
81         M6N              York   
82         M6P      West Toronto   
83         M6R      West Toronto   
84         M6S      West Toronto   
85         M7A      Queen's Park   
86         M7R       Mississauga   
87         M7Y      East Toronto   
88         M8V         Etobicoke   
89         M8W         Etobicoke   
90         M8X         Etobicoke   
91         M8Y         Etobicoke   
92         M8Z         Etobicoke   
93         M9A         Etobicoke   
94         M9B         Etobicoke   
95         M9C         Etobicoke   
96         M9L        North York   
97         M9M        North York   
98         M9N              York   
99         M9P         Etobicoke   
100        M9R         Etobicoke   
101        M9V         Etobicoke   
102        M9W         Etobicoke   

                                          Neighborhood  
0                                       Rouge, Malvern  
1               Highland Creek, Rouge Hill, Port Union  
2                    Guildwood, Morningside, West Hill  
3                                               Woburn  
4                                            Cedarbrae  
5                                  Scarborough Village  
6          East Birchmount Park, Ionview, Kennedy Park  
7                      Clairlea, Golden Mile, Oakridge  
8      Cliffcrest, Cliffside, Scarborough Village West  
9                          Birch Cliff, Cliffside West  
10   Dorset Park, Scarborough Town Centre, Wexford ...  
11                                   Maryvale, Wexford  
12                                           Agincourt  
13             Clarks Corners, Sullivan, Tam O'Shanter  
14   Agincourt North, L'Amoreaux East, Milliken, St...  
15                       L'Amoreaux West, Steeles West  
16                                         Upper Rouge  
17                                   Hillcrest Village  
18                        Fairview, Henry Farm, Oriole  
19                                     Bayview Village  
20                            Silver Hills, York Mills  
21                             Newtonbrook, Willowdale  
22                                    Willowdale South  
23                                     York Mills West  
24                                     Willowdale West  
25                                           Parkwoods  
26                                     Don Mills North  
27                    Flemingdon Park, Don Mills South  
28     Bathurst Manor, Downsview North, Wilson Heights  
29                     Northwood Park, York University  
..  

## 5. Shape of the final dataframe

In [20]:
df2.shape

(103, 3)